# 🤖 Ejercicio 1.2: Agente Básico

**Módulo 1: Fundamentos**  
**Tiempo estimado**: 12 minutos  
**Ejercicio**: 1.2

---

## 🎯 Objetivos de Aprendizaje

1. ✅ Crear herramientas con @tool
2. ✅ Usar bind_tools para dar herramientas al LLM
3. ✅ Implementar conditional_edges
4. ✅ Manejar tool_calls y ToolNode


In [ ]:
# ╔════════════════════════════════════════════════════════════╗
# ║  📋 CELL 1: SETUP Y VERIFICACIÓN                          ║
# ║  ⏱️ Tiempo: 30 segundos                                    ║
# ╚════════════════════════════════════════════════════════════╝

%pip install -q langgraph langchain-openai python-dotenv

import os
import sys
from dotenv import load_dotenv

sys.path.append(os.path.abspath('../..'))
load_dotenv()

print("╔════════════════════════════════════════╗")
print("║   SETUP VERIFICATION                   ║")
print("╚════════════════════════════════════════╝")
print(f"✅ Python {sys.version.split()[0]}")
print(f"{'✅' if os.getenv('OPENAI_API_KEY') else '❌'} OpenAI API Key")
print(f"\n🎬 Ready!\n")

## 💬 SCRIPT

> "Ahora el desafío: un agente que DECIDE qué herramientas usar.
> La clave: conditional_edges en vez de edges fijos."

In [ ]:
from typing import Annotated
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
import operator

class AgentState(TypedDict):
    messages: Annotated[list, operator.add]

# Herramientas
@tool
def search_web(query: str) -> str:
    '''Busca en internet. Usa para info actual.'''
    return f'Resultados: {query}'

@tool
def calculator(expression: str) -> str:
    '''Calcula. Usa para matemáticas.'''
    return f'Resultado: {eval(expression)}'

print('✅ Herramientas definidas')

In [ ]:
# Agente con herramientas

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

tools = [search_web, calculator]
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
llm_with_tools = llm.bind_tools(tools)

def agent_node(state):
    response = llm_with_tools.invoke(state['messages'])
    return {'messages': [response]}

def should_continue(state):
    last = state['messages'][-1]
    if hasattr(last, 'tool_calls') and last.tool_calls:
        return 'tools'
    return 'end'

# Grafo
graph = StateGraph(AgentState)
graph.add_node('agent', agent_node)
graph.add_node('tools', ToolNode(tools))

graph.add_edge(START, 'agent')
graph.add_conditional_edges('agent', should_continue, {'tools': 'tools', 'end': END})
graph.add_edge('tools', 'agent')

app = graph.compile()
print('✅ Agente compilado')

In [ ]:
# 🎬 DEMO: Diferentes inputs

tests = [
    '¿Cuánto es 25 * 4?',
    '¿Qué es LangGraph?',
    'Hola'
]

for test in tests:
    print(f'\nInput: {test}')
    result = app.invoke({'messages': [HumanMessage(content=test)]})
    
    for msg in result['messages']:
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print(f"  🔧 Usó: {msg.tool_calls[0]['name']}")
    
    print(f"  💬 Respuesta: {result['messages'][-1].content}")

---

## ✅ CHECKPOINT FINAL

### Verificar:
- [ ] El código ejecuta sin errores
- [ ] Todos entienden los conceptos clave
- [ ] Listos para el siguiente notebook

### 💬 PREGUNTA FINAL:
> "¿Alguna pregunta antes de continuar?"

---
